<a href="https://colab.research.google.com/github/evilla-19/keras_MNIST_TL_VGG19/blob/master/MNIST_fashion_TL_VGNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Following https://medium.com/nanonets/how-to-classify-fashion-images-easily-using-convnets-81e1e0019ffe and https://github.com/khanhnamle1994/fashion-mnist/blob/master/VGG19-GPU.ipynb

In [0]:
import keras
from keras.applications import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.models import Model, Sequential
from keras import models
from keras import layers
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from keras.utils import to_categorical

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split



Using TensorFlow backend.


In [0]:
# set-up to upload from google drive, part 2
import os
!pip3 install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# set-up to upload from google drive, part 3
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# actually get the files
!mkdir /content/inputData
%cd /content/inputData
download = drive.CreateFile({'id': '1TKQb-Ayc1W5aEA-sKYKPcexOwDntyDgZ'})
download.GetContentFile('fashion-mnist_test.csv')
download = drive.CreateFile({'id': '13Pye-aQ9chnOTo_dBWGha9zG2IuZq_yQ'})
download.GetContentFile('fashion-mnist_train.csv')

mkdir: cannot create directory ‘/content/inputData’: File exists
/content/inputData


In [0]:
# split the training and testing data into X (image) and Y (label) arrays

%cd /content/inputData

train_df = pd.read_csv(r'fashion-mnist_train.csv')
test_df = pd.read_csv(r'fashion-mnist_test.csv')

train_data = np.array(train_df, dtype='float32')
test_data = np.array(test_df, dtype='float32')

x_train = train_data[:, 1:]
y_train = train_data[:, 0]

x_test = test_data[:, 1:]
y_test = test_data[:, 0]

y_test = to_categorical(y_test)
y_train = to_categorical(y_train)


/content/inputData


In [0]:
# Reduce input size because else I run out of RAM memory 
x_train = x_train[0:10000, :]
x_test = x_test[0:1000, :]

y_train = y_train[0:10000 ,:]
y_test = y_test[0:1000, :]

In [0]:
x_test.shape

(999, 150, 150, 3)

In [0]:
# reduce the amount of training data because otherwise the session crashes
# x_train = x_train[0:5000,:]
# x_test = x_test[0:1000,:]
# y_train = y_train[0:5000]
# y_test = y_test[0:1000]

# Convert the training and test images into 3 channels
x_train = np.dstack([x_train] * 3)
x_test = np.dstack([x_test]*3)

# Display their new shapes
x_train.shape, x_test.shape



((10000, 784, 3), (1000, 784, 3))

In [0]:
# Reshape images as per the tensor format required by tensorflow
x_train = x_train.reshape(-1, 28,28,3)
x_test= x_test.reshape (-1,28,28,3)

# Display the new shape
x_train.shape, x_test.shape

((10000, 28, 28, 3), (1000, 28, 28, 3))

In [0]:
# Resize the images as 150 * 150 as required by VGG19
from keras.preprocessing.image import img_to_array, array_to_img


x_train = np.asarray([img_to_array(array_to_img(im, scale=False).resize((150,150))) for im in x_train])
x_test = np.asarray([img_to_array(array_to_img(im, scale=False).resize((150,150))) for im in x_test])

# Display the new shape
x_train.shape, x_test.shape

((10000, 150, 150, 3), (1000, 150, 150, 3))

In [0]:
from sklearn.model_selection import train_test_split

# Here I split original training data to sub-training (80%) and validation data (20%)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=13)

# Check the data size whether it is as per tensorflow and VGG19 requirement
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((8000, 150, 150, 3), (2000, 150, 150, 3), (8000, 10), (2000, 10))

In [0]:
# Normalise the data and change data type
x_train = x_train.astype('float32')
x_train /= 255

x_test = x_test.astype('float32')
x_test /= 255

In [0]:
# Create the base model of VGG19
vgg19 = VGG19(weights='imagenet', include_top=False, input_shape = (150, 150, 3), classes = 10)
vgg19.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
bloc

In [0]:
from keras.utils import plot_model
plot_model(vgg19, to_file='vgg19_model_plot.png', show_shapes=True, show_layer_names=True)

In [0]:
from keras.applications.vgg19 import preprocess_input

# Preprocessing the input 
x_train = preprocess_input(x_train)
x_val = preprocess_input(x_val)
x_test = preprocess_input(x_test)

In [0]:
train_features = vgg19.predict(np.array(x_train), batch_size=256, verbose=1)
test_features = vgg19.predict(np.array(x_test), batch_size=256, verbose=1)
val_features = vgg19.predict(np.array(x_val), batch_size=256, verbose=1)

2000/2000 [==============================] - 18s 9ms/step


In [0]:
# Saving the features so that they can be used for future
np.savez("train_features", train_features, y_train)
np.savez("test_features", test_features, y_test)
np.savez("val_features", val_features, y_val)

In [0]:
print(train_features.shape, "\n",  test_features.shape, "\n", val_features.shape)


(8000, 4, 4, 512) 
 (1000, 4, 4, 512) 
 (2000, 4, 4, 512)


In [0]:
# Flatten extracted features
train_features = np.reshape(train_features, (8000, 4*4*512))
test_features = np.reshape(test_features, (1000, 4*4*512))
val_features = np.reshape(val_features, (2000, 4*4*512))

In [0]:
from keras.layers import Dense, Dropout
from keras.models import Model
from keras import models
from keras import layers
from keras import optimizers

# Add Dense and Dropout layers on top of VGG19 pre-trained
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_dim=(4*4*512)))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.Dense(10, activation='softmax'))

In [0]:
# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [0]:
# !pip install tensorboardcolab
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback

tbc=TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
https://b3e7f93f.ngrok.io


In [0]:
from keras.utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [0]:
model.fit(train_features, y_train,
          batch_size=256,
          epochs=50,
          verbose=1,
          validation_data=(val_features, y_val),
          callbacks = [TensorBoardColabCallback(tbc)])

Instructions for updating:
Use tf.cast instead.
Train on 8000 samples, validate on 2000 samples
Epoch 1/50
8000/8000 [==============================] - 1s 186us/step - loss: 3.5791 - acc: 0.1984 - val_loss: 11.7687 - val_acc: 0.1125
Epoch 2/50
8000/8000 [==============================] - 1s 77us/step - loss: 3.1158 - acc: 0.3890 - val_loss: 13.1134 - val_acc: 0.1095
Epoch 3/50
8000/8000 [==============================] - 1s 74us/step - loss: 2.9163 - acc: 0.4662 - val_loss: 13.4535 - val_acc: 0.1095
Epoch 4/50
8000/8000 [==============================] - 1s 73us/step - loss: 2.7456 - acc: 0.5349 - val_loss: 13.5678 - val_acc: 0.1100
Epoch 5/50
8000/8000 [==============================] - 1s 69us/step - loss: 2.6479 - acc: 0.5593 - val_loss: 13.4752 - val_acc: 0.1100
Epoch 6/50
8000/8000 [==============================] - 1s 72us/step - loss: 2.5854 - acc: 0.5704 - val_loss: 13.6348 - val_acc: 0.1095
Epoch 7/50
8000/8000 [==============================] - 1s 69us/step - loss: 2.5228 - a

In [0]:
score = model.evaluate(test_features, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.7136822395324707
Test accuracy: 0.75


In [0]:
# get the predictions for the test data
predicted_classes = model.predict_classes(test_features)

# get the indices to be plotted
y_true = test_df[0:1000].iloc[:, 0]
correct = np.nonzero(predicted_classes==y_true)[0]
incorrect = np.nonzero(predicted_classes!=y_true)[0]

In [0]:
from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(10)]
print(classification_report(y_true, predicted_classes, target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.77      0.78      0.77        98
     Class 1       0.89      0.92      0.91        89
     Class 2       0.79      0.31      0.44       111
     Class 3       0.76      0.83      0.79       104
     Class 4       0.63      0.53      0.58        96
     Class 5       0.79      0.88      0.83        97
     Class 6       0.42      0.73      0.53       100
     Class 7       0.89      0.77      0.83       102
     Class 8       0.93      0.92      0.92        98
     Class 9       0.90      0.90      0.90       105

   micro avg       0.75      0.75      0.75      1000
   macro avg       0.78      0.76      0.75      1000
weighted avg       0.78      0.75      0.75      1000

